# Bienvenido al día 2 de la workshop de Nuclio
## 15-Enero-2025

<img src="./Pictures/ANIME.jpg" width="700">

---

# Objetivos del notebook

En este notebook vamos a construir un recomendador de animes basado en la técnica de **"Content Based Recommendation System"** desde cero.

Para ello usaremos la librería de Python `pandas` para procesar nuestro dataset y hacer la limpieza pertinente.

Si necesitan un repaso de `pandas` puede utilizar el notebook [PANDAS.ipynb](./PANDAS.ipynb) para refrescar todos los conocimientos necesarios.

---

Al final del notebook esperamos que todos los asistentes sepan:
1. En que consiste el **Content Based Recommendation System**
1. Saber preparar el dataset en el formato necesario para poder ejecutar nuestro recomendador.
1. Ejecutar un par de recomendaciones y comprobar sus resultados.
1. Saber cuales son los puntos fuertes y los puntos débiles de CBRS.


# Let's go!

---

Hacemos los imports de las principales librerías.

In [1]:
import os
import pandas as pd

Con la siguiente línea de código podemos detectar desde que ruta estamos ejecutando nuestro notebook `os.getcwd()` posteriormente con `os.path.join` juntamos la ruta de antes con la carpeta `input` (donde están nuestros ficheros y nuestros datasets.

In [2]:
PATH_INPUT_FOLDER = os.path.join(os.getcwd(), "input")

Usamos pandas para cargar los dos ficheros necesarios a Python.

In [3]:
rating = pd.read_parquet(os.path.join(PATH_INPUT_FOLDER, 'cf_rating.parquet.gzip'))
anime = pd.read_parquet(os.path.join(PATH_INPUT_FOLDER, 'cf_anime.parquet.gzip'))

Como podemos observar, tenemos muchos clientes que tiene un rating de `-1` lo más probable que esto implique que son `nulos` y deberíamos eliminarlos de nuestro dataset.

Hay varias formas de hacerlo pero lo que nosotros vamos a hacer es usar una `boolean mask`.

In [4]:
rating.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [5]:
rating.groupby(["rating"]).size()

rating
-1     1476496
 1       16649
 2       23150
 3       41453
 4      104291
 5      282806
 6      637775
 7     1375287
 8     1646019
 9     1254096
 10     955715
dtype: int64

Aplicamos nuestra `boolean mask` y podemos comprobar que los nulos han desaparecido de nuestro **DataFrame**.

In [6]:
rating = rating[rating["rating"] != -1]

In [7]:
rating.head()

,user_id,anime_id,rating
47,1,8074,10
81,1,11617,10
83,1,11757,10
101,1,15451,10
153,2,11771,10


In [8]:
rating.groupby(["rating"]).size()

rating
1       16649
2       23150
3       41453
4      104291
5      282806
6      637775
7     1375287
8     1646019
9     1254096
10     955715
dtype: int64

A su vez dentro del **DataFrame** de anime, hay muchas columnas, pero a nosotros sólo nos interesa la columna de **name** y la columna **genre**.

Vamos a usar pandas para eliminar todas las demás columnas.

In [9]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [10]:
anime = anime[["anime_id", "name", "genre"]]

In [11]:
anime.head()

,anime_id,name,genre
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural"
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili..."
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S..."
3,9253,Steins;Gate,"Sci-Fi, Thriller"
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S..."


Antes de seguir, vamos a hablar un poco más sobre nuestro recomendador.

**Content Based Recommendation Systems** se basa en:
1. Que los usuarios han usado nuestros servicios y han dejado "huella". Es decir, en nuestro caso en concreto que han dejado alguna review. Si miramos con detenimiento, vemos que en el **DataFrame** llamado `rating` tenemos una columna llamada `rating` y es la puntuación que ha dejado cada user para los animes que ha visto.
1. Por el otro lado, cada anime pertece a un género en concreto. En este nuestro caso, tenemos la columna `genre` dentro de nuestro **DataFrame** llamado `anime`. La columna `genre` es cada uno de los géneros a los que pertence algún anime.

Básicamente lo que podríamos hacer es lo siguiente:

1. "Expandir" la columna de `genre` (ahora mismo es text separado por comas) y así obtener muchas columnas.
1. Posteriormente podríamos cruzar nuestro **DataFrame** de `anime` con el **DataFrame** de `rating` por el campo de anime_id. De esta manera, nos traemos la puntuación de cada usuario y podemos saber que géneros ha puntuado y le gusta realmente.
1. Una vez que tenemos la información de antes, lo que nos faltaría por hacer es "ponderar" los géneros según la puntuación que han dejado.
1. Si tenemos toda esta información, vamos a poder empezar con nuestra recomendación.

In [12]:
ORDER_OF_COLUMNS = [
    'user_id',
    'name',
    'rating',
    'Action',
    'Adventure',
    'Cars',
    'Comedy',
    'Dementia',
    'Demons',
    'Drama',
    'Ecchi',
    'Fantasy',
    'Game',
    'Harem',
    'Hentai',
    'Historical',
    'Horror',
    'Josei',
    'Kids',
    'Magic',
    'Martial Arts',
    'Mecha',
    'Military',
    'Music',
    'Mystery',
    'Parody',
    'Police',
    'Psychological',
    'Romance',
    'Samurai',
    'School',
    'Sci-Fi',
    'Seinen',
    'Shoujo',
    'Shoujo Ai',
    'Shounen',
    'Shounen Ai',
    'Slice of Life',
    'Space',
    'Sports',
    'Super Power',
    'Supernatural',
    'Thriller',
    'Vampire',
    'Yaoi',
    'Yuri'
]

In [13]:
def multiply_genre_by_rating(df):
    '''
    Multiplies all the genre columns in the df by the rating given by the user.
    '''
    df.iloc[:, 3::] = df.iloc[:, 3::].multiply(df.iloc[:, 2], axis = 0)
    return df

In [14]:
def calculate_ponderated_score(df):
    '''
    Ponderates the genres by the user rating.
    '''
    return df.divide(df.sum(axis = 1), axis = 0)

In [16]:
ponderated_score = (
    anime                                  #  1. empezamos con nuestro df
    .set_index(["anime_id", "name"])       #  2. ponemos en el index anime_id y name para no perderlos
    ["genre"]                              #  3. seleccionamos la columna genre (es texto separado por ", ")
    .str.split(", ", expand = True)        #  4. hacemos el split y expand = True para obtener columnas en el df
    .reset_index()                         #  5. rescatamos el anime_id y name del index
    .melt(id_vars = ["anime_id", "name"])  #  6. melt para eliminar el 0, 1, 2, 3 ... del df
    [["anime_id", "name", "value"]]        #  7. seleccionamos las columnas de interés
    .dropna()                              #  8. eliminamos todos los nulos que puedan haber
    .pivot_table(                          #  9. pivot_table para poner todos los posibles géneros en columnas
        index = ["anime_id", "name"],      #  9. Nota: algunos de los animes no tendrán algún género y
        columns = "value",                 #  9. tendremos nulos
        aggfunc = len
    )
    .fillna(0)                             # 10. fillna de todos los nulos entre diferentes géneros
    .reset_index()                         # 11. sacamos anime_id y name del index (por la pivot_table)
    .merge(                                # 12. nos traemos los ratings de cada usuario
        right = rating,                    # 12. hacemos el merge por la columna de anime_id
        how = 'inner',
        on = "anime_id"
    )
    .pipe(lambda df: df[ORDER_OF_COLUMNS]) # 13. ordenamos las columnas según nos interesa
    .pipe(multiply_genre_by_rating)        # 14. multipli. cada género por el rating de cada usuario por anime
    .drop(columns = ["name", "rating"])    # 15. drop de las columnas name y rating
    .groupby(["user_id"])                  # 16. agrupamos nuestro df por user_id
    .sum()                                 # 17. sumamos todas las puntuaciones por género de cada user
    .pipe(calculate_ponderated_score)      # 18. ponderamos los scores de cada user
)

In [17]:
ponderated_score.head(10)

,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.136364,0.045455,0.000000,0.090909,0.000000,0.090909,0.000000,0.136364,0.045455,0.045455,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.0,0.0
2,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
3,0.133255,0.104363,0.000000,0.085790,0.000000,0.007665,0.068101,0.011498,0.116156,0.016509,...,0.000000,0.011498,0.000000,0.025354,0.020637,0.053656,0.009434,0.001769,0.0,0.0
5,0.085026,0.055432,0.002819,0.156665,0.000117,0.009278,0.047798,0.029477,0.049207,0.004815,...,0.000000,0.038638,0.003406,0.027481,0.020082,0.050029,0.005989,0.003641,0.0,0.0
7,0.081957,0.040153,0.000000,0.125587,0.002434,0.015123,0.032331,0.046237,0.054841,0.009995,...,0.000000,0.031636,0.000000,0.007648,0.022771,0.050408,0.007996,0.005823,0.0,0.0
8,0.133455,0.047532,0.000000,0.107861,0.000000,0.000000,0.051188,0.025594,0.060329,0.016453,...,0.000000,0.016453,0.000000,0.000000,0.047532,0.043876,0.010969,0.000000,0.0,0.0
9,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
10,0.175000,0.175000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.175000,0.112500,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
11,0.067928,0.080373,0.000000,0.093855,0.003630,0.019704,0.106041,0.009593,0.090226,0.004148,...,0.002333,0.021519,0.001815,0.004148,0.009074,0.052113,0.009334,0.003370,0.0,0.0


Hacemos este df porque lo vamos a necesitar más adelante a la hora de hacer el `dot product` con las preferencias de 1 usuario en concreto.

In [18]:
anime_genre = (
    anime                                  
    .set_index(["anime_id", "name"])       
    ["genre"]                              
    .str.split(", ", expand = True)        
    .reset_index()                         
    .melt(id_vars = ["anime_id", "name"])  
    [["anime_id", "name", "value"]]        
    .dropna()                              
    .pivot_table(                          
        index = ["anime_id", "name"],      
        columns = "value",                 
        aggfunc = len
    )
    .fillna(0)                             
    .reset_index()
    .set_index(["anime_id", "name"])
)

In [19]:
anime_genre.head()

,value,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
anime_id,name,,,,,,,,,,,,,,,,,,,,,
1,Cowboy Bebop,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Cowboy Bebop: Tengoku no Tobira,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Trigun,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Witch Hunter Robin,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,Beet the Vandel Buster,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


# Parte de Inferencia

Ya tenemos calculado para cada usuario los géneros que más le gustan.

Ahora vamos a poder empezar a hacer su recomendación.

Seleccionamos un usuario al azar.

In [20]:
random_user = pd.Series(rating["user_id"].unique()).sample().iloc[0]
random_user = 54273 # comentar esta línea de código para seleccionar un user al azar

In [21]:
random_user

54273

Filtramos únicamente las preferencias ponderadas de nuestro usuario.

In [22]:
random_user_ponderated_score = ponderated_score[ponderated_score.index.isin([random_user])]

In [23]:
random_user_ponderated_score

,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
user_id,,,,,,,,,,,,,,,,,,,,,
54273,0.130603,0.062401,0.00668,0.102303,0.0,0.010722,0.076288,0.010722,0.045175,0.012304,...,0.0,0.015996,0.002285,0.077518,0.03287,0.044999,0.002812,0.003691,0.0,0.0


Ahora vamos a ver que géneros más le gustan a este usuario.

In [24]:
top_10_favorite_genres = (
    random_user_ponderated_score
    .melt()
    .sort_values("value", ascending = False)
    .head(10)
)

In [25]:
top_10_favorite_genres

,variable,value
0,Action,0.130603
32,Shounen,0.130603
3,Comedy,0.102303
36,Sports,0.077518
6,Drama,0.076288
1,Adventure,0.062401
25,Romance,0.050448
27,School,0.049569
8,Fantasy,0.045175
38,Supernatural,0.044999


Vamos a seleccionar los animes que el usuario ya ha visto (es para eliminarlos más adelante).

In [26]:
animes_watched_by_random_user = rating[rating["user_id"] == random_user]["anime_id"].values.tolist()

Ahora lo que vamos a hacer es multiplicar las preferencias de nuestro usuario por todos y cada de los géneros de todos y cada uno de los animes. Esta operación se llama `dot product`. Esto básicamente nos permitirá calcular un score para cada anime que luego nos permitirá ordenar y afinar nuestra recomendación.

In [31]:
recommendation = (
    random_user_ponderated_score
    .dot(anime_genre.T)
    .T
    .rename(columns = {random_user:"score"})
    .sort_values(by = "score", ascending = False)
    .reset_index()
    .pipe(lambda df: df[-df["anime_id"].isin(animes_watched_by_random_user)])
    .merge(
        right = anime_genre.reset_index(),
        on = ["anime_id", "name"]
    )
)

In [32]:
recommendation.head()

,anime_id,name,score,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
0,231,Asagiri no Miko,0.668307,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,1186,Battle Athletess Daiundoukai (TV),0.654069,1.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,573,Saber Marionette J,0.595887,1.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1397,Macross 7,0.587098,1.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,808,Bakuretsu Hunters OVA,0.586219,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [33]:
top_10_favorite_genres_list = top_10_favorite_genres["variable"].values.tolist()

In [34]:
recommendation[["anime_id", "name", "score"] + top_10_favorite_genres_list].head()

,anime_id,name,score,Action,Shounen,Comedy,Sports,Drama,Adventure,Romance,School,Fantasy,Supernatural
0,231,Asagiri no Miko,0.668307,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
1,1186,Battle Athletess Daiundoukai (TV),0.654069,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0
2,573,Saber Marionette J,0.595887,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
3,1397,Macross 7,0.587098,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
4,808,Bakuretsu Hunters OVA,0.586219,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0


1. [La recomendación top para nuestro usuario es este anime: "Asagiri no Miko"](https://www.imdb.com/es-es/title/tt1071154/)<br>
1. [Seguida de: Battle Athletess Daiundoukai (TV)](https://www.imdb.com/es-es/title/tt13009154/)<br>
1. [Y cierra el top 3: Saber Marionette J](https://www.imdb.com/es-es/title/tt0211605/)<br>

Algunas consideraciones importantes:
1. Si nuestro usuario **no tiene historial** no podemos hacerle ninguna recomendación (problema conocido como **inicio frío**).
1. Nuestra CBRS es una recomendación que necesita "cargar" toda la información en memoria (modelo **memory based**). Esto puede ser costoso si tengo miles de millones de registros.
1. Nuestra recomendación es "incapaz" de sugerirle a nuestro usuario algo que no pertenezca a los géneros que le gustan. Esto puede ser un problema porque el **usuario seguirá consumiendo "siempre lo mismo"** (efecto burbuja informativa).
1. Las características deben estar bien definidas para poder hacer una recomendación como la de antes. Es decir, si los géneros no están bien definidos, la recomendación no será buena (**calidad de los datos**).
1. **A pesar de las carencias de antes, el CBRS es una técnica muy sencilla y muy fácil de implementar en las fases iniciales de construir un recomendador.**